In [ ]:
import nltk
nltk.download('reuters')
from nltk.corpus import reuters
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

[nltk_data] Downloading package reuters to /root/nltk_data...


In [ ]:
def tokenise(line):
	punc = [',', '.', '"', "'", '/', '*', ',', '?', '!', '-', '\n', '“', '”', '_', '&', '\ufeff', '&', ';', ":"]
	for el in line:
		if el in punc:
			line = line.replace(el, "")
	
	line=line.lower()
	return line.split()
	raise NotImplementedError

tokenise(reuters.raw(reuters.fileids()[10787]))[:5]

['ltaha', 'automotive', 'technologies', 'corp>', 'year']

In [ ]:
nltk.download('stopwords')
def remove_stop_words(list_of_words):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    ans=[word for word in list_of_words if word not in stopwords]
    return ans
    raise NotImplementedError

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
raw=[]
for i in range(10787):
  doc=tokenise(reuters.raw(reuters.fileids()[i]))
  raw.append(doc)
print(len(raw),raw[0][:5])

10787 ['asian', 'exporters', 'fear', 'damage', 'from']


In [ ]:
tokens=[]
for i in range(len(raw)):
 for x in raw[i]:
  tokens.append(x)
print(len(tokens),tokens[:5])#total words=52933

1376160 ['asian', 'exporters', 'fear', 'damage', 'from']


In [ ]:
words=remove_stop_words(tokens)
len(words),words[90]

(941414, 'major')

In [ ]:
nwo=[]
for w in words:
    if not w.isdigit():
      nwo.append(w)
len(nwo)

856922

In [ ]:
nwo[0]

'asian'

In [ ]:
len(tokens)-len(words)

434746

In [ ]:
df_words=pd.DataFrame(nwo)
df_words.to_csv("new_words.csv")

In [ ]:
vocab=set()
for i in range(len(words)):
  vocab.add(words[i])
len(vocab)

52800

In [ ]:
vocab1=set()
for i in range(len(raw)):
 for x in raw[i]:
  vocab1.add(x)
print(len(vocab1))#distinct words=52933


52933


In [ ]:
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}
print(word_to_ix["japan"],ix_to_word[14141])

51649 northeasts


In [ ]:
def svd(mat):
 u, s, v = svds(mat, k=2)
 return u.dot(np.diag(np.sqrt(s)))

In [ ]:
ix_to_word[0],word_to_ix['cornerstone']

('79745', 16682)

In [ ]:
k=3
mat3= np.zeros((len(vocab), len(vocab)))
for i in range(len(words)):
  for j in range(max(0, i-k), min(len(words), i+k+1)):
    if i != j:
      mat3[word_to_ix[words[i]],word_to_ix[words[j]]] += 1

In [ ]:
emb3=svd(mat3)

In [ ]:
emb3[word_to_ix["japan"]]

array([-0.05642647, -0.53613121])

In [ ]:
def cos_similar(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_mat = cosine_similarity(emb3)

In [ ]:
cos_similar(embed_5[word_to_ix['india']],embed_5[word_to_ix['rupee']])#,cos_similar(embed_3[word_to_ix['india']],embed_3[word_to_ix['rupee']])

In [ ]:
s1=cos_similar(emb3[word_to_ix['india']],emb3[word_to_ix['rupee']])
s2=cos_similar(emb3[word_to_ix['rupee']],emb3[word_to_ix['japan']])
s1,s2

(0.8702674049718417, 0.8989430418843285)

In [ ]:
emb3[word_to_ix['leg']]

In [ ]:
import matplotlib.pyplot as plt
def plot_vector(emb,col):
  i,j=[emb[0]],[emb[1]]
  plt.quiver([0], [0], i, j, color=col, units='xy', scale=1)


In [ ]:
plot_vector(embed[word_to_ix['india']],'b')
plot_vector(embed[word_to_ix['japan']],'r')
plot_vector(embed[word_to_ix['tokyo']],'g')

In [ ]:
x1=embed[word_to_ix['leg']]
x2=embed[word_to_ix['kick']]
x3=embed[word_to_ix['hand']]
x4=x2-x1+x3

In [ ]:
x1

In [ ]:
plot_vector(x1,'r')
plot_vector(x2,'b')
plot_vector(x3,'g')

In [ ]:
em = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [1.0, 1.1, 1.2]])
max_index = np.argmax(em)
max_index

In [ ]:
embed[14141],np.argmax(embed[14141]),embed[word_to_ix["japan"]]

In [ ]:
ix_to_word[embed[14141]]

In [ ]:
def embed_to_word(embed_vector):
    max_index = np.argmax(embed_vector)
    word = list(word_to_ix.keys())[list(word_to_ix.values()).index(max_index)]
    return word

In [ ]:
c=0
def most_similar(emb,embed):
  #global c
  s=cos_similar(emb,embed[0])
  for i in range(len(emb)):
    if(cos_similar(emb,embed[i])>=s):c=i
  return ix_to_word[c] 

In [ ]:
most_similar(emb3[word_to_ix["japan"]],emb3)

'ltdbs'

In [ ]:
emb3[word_to_ix["ltdbs"]]

array([-0.01358681, -0.04105824])

In [ ]:
most_similar(emb)

In [ ]:
embed[word_to_ix["lowdensity"]],x1,x2